<a href="https://colab.research.google.com/github/julienkay/com.doji.chromatic/blob/master/tools/matplotlib_colormap_png_export.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Generate PNGs

import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from PIL import Image, ImageDraw
import numpy as np
import os

# Function to create a colormap image
def create_colormap_image(cmap, filename):
    gradient = cmap(np.linspace(0, 1, 256))
    gradient = (gradient * 255).astype(int)

    img = Image.new('RGB', (256, 1))
    draw = ImageDraw.Draw(img)

    for i, color in enumerate(gradient):
        draw.line((i, 0, i, 30), fill=tuple(color))

    img.save(filename)

os.makedirs('pngs', exist_ok=True)

# Get a list of all available colormaps
all_colormaps = [m for m in plt.colormaps() if not m.endswith("_r")]

# Iterate through colormaps and save PNG files
for cmap_name in all_colormaps:
    cmap = plt.get_cmap(cmap_name)
    create_colormap_image(cmap, f'pngs/{cmap_name}.png')

print("Colormap images saved.")


In [ ]:
!zip -r /content/pngs.zip /content/pngs

# @title Download PNGs
from google.colab import files

from google.colab import files
files.download("/content/pngs.zip")

In [ ]:
# @title Generate C# classes/files

output_file1 = 'ColormapProperties.cs'
output_file2 = 'Colormaps.cs'

header = '''/*
 * ----------------------------------------------------------------
 * <auto-generated>
 * This file is auto-generated and should not be modified manually.
 * </auto-generated>
 * ----------------------------------------------------------------
 */

using UnityEngine;

namespace Chromatic {

    public static partial class ColorMaps {
'''

footer = '''    }
}'''

# Generate C# properties for each colormap
with open(output_file1, 'w') as file:

    file.write(header)

    for cmap_name in all_colormaps:
        # Create C# property code with the current colormap name
        property_code = f'''
        public static Texture2D {cmap_name} => _{cmap_name} ?? (_{cmap_name} = Resources.Load<Texture2D>("{cmap_name}"));
        private static Texture2D _{cmap_name};\n'''

        file.write(property_code)

    file.write(footer)

print(f'C# code has been written to {output_file1}')


# Generate a C# list that contains all colormaps
with open(output_file2, 'w') as file:

    file.write(header)

    maps = []

    for cmap_name in all_colormaps:
        maps.append(f'''            {cmap_name}''')

    maps_array = ',\n'.join(maps)

    file.write(f'''\n        public static Texture2D[] Colormaps = new Texture2D[] {{\n{maps_array}\n        }};\n''')

    file.write(footer)

print(f'C# code has been written to {output_file2}')